In [17]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import pymongo

In [18]:

# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.indeedDB

In [2]:
URL = "https://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [3]:
r = requests.get(URL)

soup = BeautifulSoup(r.text, 'html.parser')

jobs = soup.find_all(name='div', attrs={'class':"result"})

In [4]:
def extract_company_from_result(html):
    try:
        return html.find("span",{"class":"company"}).text
    except:
        return None

In [5]:
def extract_salary_from_result(html):
    try:
        return html.find("span",{"class":"no-wrap"}).text
    except:
        return None

In [6]:
def extract_location_from_result(html):   
    try:
        return html.find("span",{'class':'location'}).text
    except:
        return None

In [7]:
def extract_job_from_result(html):
    try:
        return html.find("a").text
    except:
        return None

In [8]:
def extract_date_from_result(html):   
    try:
        return html.find("span",{'class':'date'}).text
    except:
        return None

In [9]:
url_template = "https://www.indeed.com/jobs?q=data+scientist&1={}&start=10"
max_results_per_city = 2000
results = []

for city in ['New York','New Jersey']:
    for start in tqdm(range(0, max_results_per_city, 10)):
        r1 = requests.get(url_template.format(city,start))

        soup = BeautifulSoup(r1.text, 'html.parser')

        jobs = soup.find_all(name='div', attrs={'class':"result"}) # Grab the results from the request (as above)
        results.extend(jobs) # Append to the full set of results

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:27<00:00,  2.28it/s]


In [11]:
df = pd.DataFrame({"job_title":extract_job_from_result(result),
                   "company":extract_company_from_result(result),
                   "location":extract_location_from_result(result),
                   "salary":extract_salary_from_result(result),
                   "posted":extract_date_from_result(result), 
                   "date_extracted": pd.to_datetime('today')} for result in results)

In [12]:
df.replace(to_replace="\n",value="",regex=True,inplace=True)
df.head(50)

,job_title,company,location,salary,posted,date_extracted
0,,Triplebyte,None,"$150,000 - $225,000 a year",16 days ago,2020-04-19 17:33:47.677877
1,Sr. Data Scientist/Machine Learning Engineer,aikiu,None,"$70,000 - $110,000 a year",30+ days ago,2020-04-19 17:33:47.678875
2,Data Scientist,Seen by Indeed,None,None,23 days ago,2020-04-19 17:33:47.679872
3,Senior Data Scientist (Remote-friendly),Noom Inc.,None,None,30+ days ago,2020-04-19 17:33:47.680870
4,Data Scientist,Global Fishing Watch,None,$45 - $65 an hour,16 days ago,2020-04-19 17:33:47.680870
5,Data Scientist,ClearOne Advantage,None,"$70,000 - $80,000 a year",27 days ago,2020-04-19 17:33:47.681867
6,Data Scientist,Northrop Grumman,None,None,30+ days ago,2020-04-19 17:33:47.682864
7,Data Scientist,Andor Health,"Orlando, FL 32811 (Eagles Nest area)",None,Today,2020-04-19 17:33:47.682864
8,Lead Data Scientist,CPNET,"Harrisburg, PA","$85,000 - $95,000 a year",16 days ago,2020-04-19 17:33:47.683862
9,Data Scientist/Analytics Consultant,Deloitte,"Arlington, VA 22209 (North Rosslyn area)",None,5 days ago,2020-04-19 17:33:47.684858


In [13]:
df.to_csv("indeed_jobs.csv")

In [30]:
df_raw = df
df_raw['days posted'] = ''
df_raw

,job_title,company,location,salary,posted,date_extracted,days posted
0,,Triplebyte,None,"$150,000 - $225,000 a year",16 days ago,2020-04-19 17:33:47.677877,
1,Sr. Data Scientist/Machine Learning Engineer,aikiu,None,"$70,000 - $110,000 a year",30+ days ago,2020-04-19 17:33:47.678875,
2,Data Scientist,Seen by Indeed,None,None,23 days ago,2020-04-19 17:33:47.679872,
3,Senior Data Scientist (Remote-friendly),Noom Inc.,None,None,30+ days ago,2020-04-19 17:33:47.680870,
4,Data Scientist,Global Fishing Watch,None,$45 - $65 an hour,16 days ago,2020-04-19 17:33:47.680870,
...,...,...,...,...,...,...,...
7595,Junior Data Scientist | TS/SCI,Connect Talent Solutions,"Springfield, MA",None,30+ days ago,2020-04-19 17:33:51.965810,
7596,Data Scientist,West Creek Financial,"Washington, DC",None,1 day ago,2020-04-19 17:33:51.966808,
7597,Data Scientist,Source Enterprises,"New York, NY",None,30+ days ago,2020-04-19 17:33:51.966808,
7598,Data Scientist,Big Zeta,None,"$100,000 a year",30+ days ago,2020-04-19 17:33:51.967808,


In [ ]:
df[df['posted'].str.contains('day ago')] 

In [2]:
indeed_raw = pd.read_csv("indeed_jobs.csv")
indeed_raw.head(10)

,Unnamed: 0,job_title,company,location,salary,posted,date_extracted
0,0,NaN,Triplebyte,NaN,"$150,000 - $225,000 a year",16 days ago,2020-04-19 17:33:47.677877
1,1,Sr. Data Scientist/Machine Learning Engineer,aikiu,NaN,"$70,000 - $110,000 a year",30+ days ago,2020-04-19 17:33:47.678875
2,2,Data Scientist,Seen by Indeed,NaN,NaN,23 days ago,2020-04-19 17:33:47.679872
3,3,Senior Data Scientist (Remote-friendly),Noom Inc.,NaN,NaN,30+ days ago,2020-04-19 17:33:47.680870
4,4,Data Scientist,Global Fishing Watch,NaN,$45 - $65 an hour,16 days ago,2020-04-19 17:33:47.680870
5,5,Data Scientist,ClearOne Advantage,NaN,"$70,000 - $80,000 a year",27 days ago,2020-04-19 17:33:47.681867
6,6,Data Scientist,Northrop Grumman,NaN,NaN,30+ days ago,2020-04-19 17:33:47.682864
7,7,Data Scientist,Andor Health,"Orlando, FL 32811 (Eagles Nest area)",NaN,Today,2020-04-19 17:33:47.682864
8,8,Lead Data Scientist,CPNET,"Harrisburg, PA","$85,000 - $95,000 a year",16 days ago,2020-04-19 17:33:47.683862
9,9,Data Scientist/Analytics Consultant,Deloitte,"Arlington, VA 22209 (North Rosslyn area)",NaN,5 days ago,2020-04-19 17:33:47.684858


In [6]:
df = indeed_raw.dropna()
df.count()

Unnamed: 0        993
job_title         993
company           993
location          993
salary            993
posted            993
date_extracted    993
dtype: int64

In [7]:
df[['posted','etc','etc.']] = df.posted.apply(lambda x: pd.Series(str(x).split(" ")))

C:\Users\fraza\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [8]:
df[['posted','1']] = df.posted.apply(lambda x: pd.Series(str(x).split("+")))

In [12]:
df = df.drop(['etc','etc.','1'], axis=1)

In [13]:
df['posted'].replace(to_replace ="Just", value =0)
df

,Unnamed: 0,job_title,company,location,salary,posted,date_extracted
8,8,Lead Data Scientist,CPNET,"Harrisburg, PA","$85,000 - $95,000 a year",16,2020-04-19 17:33:47.683862
11,11,Data Scientist I - Remote,HMS Health Management Systems,"Irving, TX",$50 an hour,17,2020-04-19 17:33:47.685856
13,13,Data Scientist,Getronics,"Atlanta, GA",$50 - $55 an hour,19,2020-04-19 17:33:47.686853
24,24,Lead Data Scientist,CPNET,"Harrisburg, PA","$85,000 - $95,000 a year",16,2020-04-19 17:33:47.693835
27,27,Data Scientist I - Remote,HMS Health Management Systems,"Irving, TX",$50 an hour,17,2020-04-19 17:33:47.695829
...,...,...,...,...,...,...,...
7570,7570,Data Science Intern,Dolphin,"New York, NY 10014 (SoHo area)",$25 an hour,30,2020-04-19 17:33:51.946860
7574,7574,Data Scientist,"Big Data Federation, Inc.","Santa Clara, CA 95050","$100,000 - $120,000 a year",17,2020-04-19 17:33:51.953842
7589,7589,Lead Data Scientist,CPNET,"Harrisburg, PA","$85,000 - $95,000 a year",16,2020-04-19 17:33:51.961820
7592,7592,Data Scientist I - Remote,HMS Health Management Systems,"Irving, TX",$50 an hour,17,2020-04-19 17:33:51.963822


In [15]:
df = df.drop(['Unnamed: 0'], axis=1)

In [29]:
df

,job_title,company,location,salary,posted,date_extracted
8,Lead Data Scientist,CPNET,"Harrisburg, PA","$85,000 - $95,000 a year",16,2020-04-19 17:33:47.683862
11,Data Scientist I - Remote,HMS Health Management Systems,"Irving, TX",$50 an hour,17,2020-04-19 17:33:47.685856
13,Data Scientist,Getronics,"Atlanta, GA",$50 - $55 an hour,19,2020-04-19 17:33:47.686853
24,Lead Data Scientist,CPNET,"Harrisburg, PA","$85,000 - $95,000 a year",16,2020-04-19 17:33:47.693835
27,Data Scientist I - Remote,HMS Health Management Systems,"Irving, TX",$50 an hour,17,2020-04-19 17:33:47.695829
...,...,...,...,...,...,...
7570,Data Science Intern,Dolphin,"New York, NY 10014 (SoHo area)",$25 an hour,30,2020-04-19 17:33:51.946860
7574,Data Scientist,"Big Data Federation, Inc.","Santa Clara, CA 95050","$100,000 - $120,000 a year",17,2020-04-19 17:33:51.953842
7589,Lead Data Scientist,CPNET,"Harrisburg, PA","$85,000 - $95,000 a year",16,2020-04-19 17:33:51.961820
7592,Data Scientist I - Remote,HMS Health Management Systems,"Irving, TX",$50 an hour,17,2020-04-19 17:33:51.963822


In [31]:
df['date_extracted']

8       2020-04-19 17:33:47.683862
11      2020-04-19 17:33:47.685856
13      2020-04-19 17:33:47.686853
24      2020-04-19 17:33:47.693835
27      2020-04-19 17:33:47.695829
                   ...            
7570    2020-04-19 17:33:51.946860
7574    2020-04-19 17:33:51.953842
7589    2020-04-19 17:33:51.961820
7592    2020-04-19 17:33:51.963822
7594    2020-04-19 17:33:51.965810
Name: date_extracted, Length: 993, dtype: object

In [32]:
# loop through DATAFRAME and populate Mongo
for index, row in df.iterrows():
#     print(row['date'].dt.date)
    # Insert a document into the 'students' collection
    db.jobs.insert_one(
        {
            'job_title': row['job_title'],
            'company': row['company'] ,
            'location': row['location'] ,
            'salary':  row['salary'] ,
            'posted':  row['posted'] ,
            'date_extracted':  row['date_extracted'] 


        }
     )

# # query the classroom collection
# classroom = db.classroom.find()

# # see change in collection
# for student in classroom:
#     print(student)